In [3]:
%autosave 60

Autosaving every 60 seconds


In [4]:
import os
import sys
import time
import requests
import datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import *
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
months = dict(zip(['августа',
 'апреля',  'декабря',  'июля',
 'июня',  'марта',  'мая',
 'ноября',  'октября',  'сентября',
 'февраля',  'января'], [8, 4, 12, 7, 6, 3, 5, 11, 10, 9, 2, 1]))

In [6]:
s=Service(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\98.0.4758.102\chromedriver.exe] found in cache


In [7]:
# !"{sys.executable}" -m pip install --upgrade webdriver_manager

In [8]:
# # Read already gathered data

# df_first_articles = pd.DataFrame()
# for i in os.listdir(os.getcwd()):
#     if i.find(".csv") > -1:
#         df_temp = pd.read_csv(i, sep=";")        
#         df_first_articles = pd.concat([df_first_articles, df_temp], sort=True)
# df_first_articles.columns=["num", "datestr", "type", "header", "text", "url"]
# df_first_articles = df_first_articles.sort_values(by=['num']).reset_index(drop=True)

## Часть 1: парсим новый сайт ЦБ
### Выгрузка пресс-релизов, новостей и выступлений
https://www.cbr.ru/

In [105]:
# Create two windows: one with the main site and one for sub-windows

url = "https://www.cbr.ru/"

driver_base = webdriver.Chrome(service=s)
driver_base.maximize_window()
driver_base.get(url)
parent_tab = driver_base.window_handles[0]

driver_runner1 = webdriver.Chrome(service=s)
driver_runner1.maximize_window()

In [106]:
# Собираем в 4 этапа: 
# Новости
# Пресс-релизы
# Интервью - не нужно
# Выступления

# Так как приходится кликать разделы вручную :(
# Запустить кликер -> считать категорию -> обновить страницу -> повторить

In [110]:
def parse_by_type(articles_headers, articles_links):
    counter = 0
    articles = []
    exception_articles = []
    for header, link in tqdm(zip(articles_headers, articles_links)):
        counter += 1
        try:
            article_type = header.text.split("\n")[-1]
            article_url = link.get_attribute(name='href')
            driver_runner1.get(article_url)
            article_date = driver_runner1.find_elements(By.TAG_NAME, value='.news-info-line_date')[0].text
            article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
            articles.append([counter, article_date, article_type, link.text, "\n".join(article_text).strip(), article_url])
        except Exception as e:
            print("Problem!")
            exception_articles.append((header, link))  
    return articles, exception_articles

In [207]:
%%time

# Clicker!
# Click all buttons "Загрузить ещё" to get all the news in one window

try:
    while True:
        buttons = driver_base.find_elements(By.TAG_NAME, value='#_buttonLoadNextEvt')
        button = [i for i in buttons if i.text == 'Загрузить еще']
        button[0].click()
except Exception as e:
    print(e)
    print("End reached!")

list index out of range
End reached!
Wall time: 1.44 s


In [108]:
news_headers = driver_base.find_elements(By.TAG_NAME, value='.news_info')
news_links = driver_base.find_elements(By.TAG_NAME, value='.news_title')
print(len(news_headers), len(news_links))

1020 1020


In [111]:
news_articles, news_exc = parse_by_type(news_headers, news_links)

df_news_new = pd.DataFrame(news_articles)
df_news_new = df_news_new[df_news_new[2] != ""].reset_index(drop=True)
df_news_new[0] = df_news_new[0] - 20

1020it [13:22,  1.27it/s]


In [153]:
pr_headers = driver_base.find_elements(By.TAG_NAME, value='.news_info')
pr_links = driver_base.find_elements(By.TAG_NAME, value='.news_title')
print(len(pr_headers), len(pr_links))

1020 1020


In [201]:
pr_articles, pr_exc = parse_by_type(pr_headers, pr_links)

df_pr_new = pd.DataFrame(pr_articles)
df_pr_new = df_pr_new[df_pr_new[2] != ""].reset_index(drop=True)
df_pr_new[0] = df_pr_new[0] - 20

In [205]:
a = df_pr_new.reset_index().iloc[720:]
a[a['index'] != a[0] -2]
df_pr_new.iloc[740:760]
# header, link = pr_exc[1]

# article_type = header.text.split("\n")[-1]
# article_url = link.get_attribute(name='href')
# driver_runner1.get(article_url)
# # article_date = driver_runner1.find_elements(By.TAG_NAME, value='.news-info-line_date')[0].text
# article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
# article_date = "11 марта 2020 года"

# df_pr_new = df_pr_new.append([[760, article_date, article_type, link.text, "\n".join(article_text).strip(), article_url]])
# df_pr_new = df_pr_new.sort_values(by=[0]).reset_index(drop=True)

In [208]:
speech_headers = driver_base.find_elements(By.TAG_NAME, value='.news_info')
speech_links = driver_base.find_elements(By.TAG_NAME, value='.news_title')
print(len(speech_headers), len(speech_links))

153 153


In [209]:
speech_articles, speech_exc = parse_by_type(speech_headers, speech_links)

153it [05:55,  2.32s/it]


In [213]:
df_speech_new = pd.DataFrame(speech_articles)
df_speech_new = df_speech_new[df_speech_new[2] != ""].reset_index(drop=True)
df_speech_new[0] = df_speech_new[0] - 20

In [ ]:
df_new = pd.DataFrame()
for df_articles_temp, name in zip([df_news_new, df_pr_new, df_speech_new], ["news","press_rel","speech"]):
    df_articles_temp.to_csv(f"{name}_new.csv", 
                            encoding="utf-8", sep=";", index=False)
    df_new = pd.concat([df_new, df_articles_temp])
df_new.columns=["num", "datestr", "type", "header", "text", "url"]
df_new = df_new.drop(columns=['num'])
df_new = df_new.reset_index(drop=True)

### Здесь можно забрать готовые данные

In [9]:
df_new = pd.DataFrame()
for name in ["news","press_rel","speech"]:
    df_articles_temp = pd.read_csv(f"{name}_new.csv", 
                            encoding="utf-8", sep=";")
    df_new = pd.concat([df_new, df_articles_temp])
df_new.columns=["num", "datestr", "type", "header", "text", "url"]
df_new = df_new.drop(columns=['num'])
df_new = df_new.reset_index(drop=True)

df_new['date'] = [f"{i[0]}-{months[i[1]]}-{i[2]}" for i in df_new.datestr.str.split().tolist()]
df_new['date'] = pd.to_datetime(df_new['date'], format='%d-%m-%Y')

In [10]:
df_new.head()

,datestr,type,header,text,url,date
0,21 февраля 2022 года,Новость,Объем средств на счетах эскроу в долевом строи...,"Объем средств, размещенных участниками долевог...",https://www.cbr.ru/press/event/?id=12705,2022-02-21
1,17 февраля 2022 года,Новость,Мониторинг отраслевых финансовых потоков: рост...,Динамика отраслевых финансовых потоков указыва...,https://www.cbr.ru/press/event/?id=12702,2022-02-17
2,17 февраля 2022 года,Новость,В январе инфляция ускорилась в большинстве рег...,Годовая инфляция в январе находилась в диапазо...,https://www.cbr.ru/press/event/?id=12701,2022-02-17
3,17 февраля 2022 года,Новость,Банк России разработал детскую игру о признака...,Игра «Космический детектив: тайна семи банкнот...,https://www.cbr.ru/press/event/?id=12698,2022-02-17
4,16 февраля 2022 года,Новость,"В 2021 году Банк России выявил почти 2,7 тысяч...",Среди них 871 финансовая пирамида (в 2020 году...,https://www.cbr.ru/press/event/?id=12695,2022-02-16


In [11]:
df_new[df_new.type == "Пресс-релиз"].min()

datestr                                  1 августа 2019 года
type                                             Пресс-релиз
header     А. Данилов назначен директором Департамента об...
text       1. В целях применения абзаца второго и третьег...
url        https://www.cbr.ru/press/pr/?file=01022021_084...
date                                     2019-07-17 00:00:00
dtype: object

## Часть 2: парсим старый сайт ЦБ
### Выгрузка пресс-релизов, новостей и выступлений
http://old.cbr.ru/press/

In [93]:
url = "http://old.cbr.ru/press/month_archive/press_archive/"

driver_base = webdriver.Chrome(service=s)
driver_base.maximize_window()
driver_base.get(url)
parent_tab = driver_base.window_handles[0]

driver_runner1 = webdriver.Chrome(service=s)
driver_runner1.maximize_window()

In [94]:
start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2019, 7, 16)

In [1]:
def half_parser(cur_date, end_date, counter):
    exc_articles = []
    good_articles = []
    
    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[0].send_keys("\b\b\b\b\b\b\b\b\b\b" + cur_date.strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[1].send_keys("\b\b\b\b\b\b\b\b\b\b" + end_date.strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='#UniDbQuery_searchbutton')[0].click()

    articles_dates1 = driver_base.find_elements(By.TAG_NAME, value='.source')
    articles_links1 = driver_base.find_elements(By.TAG_NAME, value='a')[15:-10]
    for article_date, link in tqdm(zip(articles_dates, articles_links)):
        if 
        counter += 1
        try:
            article_type = "Пресс-релиз"
            article_url = link.get_attribute(name='href')
            driver_runner1.get(article_url)
            article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
            good_articles.append([counter, article_date.text, article_type, link.text, "\n".join(article_text).strip(), article_url])
        except Exception as e:
            print("Problem!")
            exc_articles.append((article_date, link)) 
    return good_articles, exc_articles, counter


counter = 0
cur_date = start_date
temp_end = cur_date + relativedelta(months=+1) - relativedelta(days=+1)
exc_articles = []
good_articles = []
while temp_end < end_date:
    print(cur_date, " - ", temp_end)
    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[0].send_keys("\b\b\b\b\b\b\b\b\b\b" + cur_date.strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[1].send_keys("\b\b\b\b\b\b\b\b\b\b" + 
                                                                                (cur_date + relativedelta(days=+14)).strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='#UniDbQuery_searchbutton')[0].click()

    articles_dates1 = driver_base.find_elements(By.TAG_NAME, value='.source')
    articles_links1 = driver_base.find_elements(By.TAG_NAME, value='a')[15:-10]

    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[0].send_keys("\b\b\b\b\b\b\b\b\b\b" + 
                                                                                (cur_date + relativedelta(days=+15)).strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[1].send_keys("\b\b\b\b\b\b\b\b\b\b" + temp_end.strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='#UniDbQuery_searchbutton')[0].click()

    articles_dates2 = driver_base.find_elements(By.TAG_NAME, value='.source')
    articles_links2 = driver_base.find_elements(By.TAG_NAME, value='a')[15:-10]

    articles_dates = articles_dates1 + articles_dates2
    articles_links = articles_links1 + articles_links2
    
    for article_date, link in tqdm(zip(articles_dates, articles_links)):
        if 
        counter += 1
        try:
            article_type = "Пресс-релиз"
            article_url = link.get_attribute(name='href')
            driver_runner1.get(article_url)
            article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
            good_articles.append([counter, article_date.text, article_type, link.text, "\n".join(article_text).strip(), article_url])
        except Exception as e:
            print("Problem!")
            exc_articles.append((article_date, link))  

    if cur_date.month == 2:
        pd.DataFrame(good_articles).to_csv(f"{cur_date}_old.csv", encoding="utf-8", sep=";", index=False)    

    cur_date = cur_date + relativedelta(months=+1)
    temp_end = cur_date + relativedelta(months=+1) - relativedelta(days=+1)
    temp_end = min([temp_end, end_date])

In [115]:
articles_links1[0].text

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=98.0.4758.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x006069A3+2582947]
	Ordinal0 [0x0059A6D1+2139857]
	Ordinal0 [0x00493A98+1063576]
	Ordinal0 [0x004962B7+1073847]
	Ordinal0 [0x0049617E+1073534]
	Ordinal0 [0x004963F0+1074160]
	Ordinal0 [0x004BB8E0+1226976]
	Ordinal0 [0x004D854C+1344844]
	Ordinal0 [0x004B6524+1205540]
	Ordinal0 [0x004D86A4+1345188]
	Ordinal0 [0x004E834A+1409866]
	Ordinal0 [0x004D8366+1344358]
	Ordinal0 [0x004B5176+1200502]
	Ordinal0 [0x004B6066+1204326]
	GetHandleVerifier [0x007ABE02+1675858]
	GetHandleVerifier [0x0086036C+2414524]
	GetHandleVerifier [0x0069BB01+560977]
	GetHandleVerifier [0x0069A8D3+556323]
	Ordinal0 [0x005A020E+2163214]
	Ordinal0 [0x005A5078+2183288]
	Ordinal0 [0x005A51C0+2183616]
	Ordinal0 [0x005AEE1C+2223644]
	BaseThreadInitThunk [0x7702FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77407A9E+286]
	RtlGetAppContainerNam

In [33]:
df_pr_old = pd.read_csv("2020-02-01_old.csv",sep=";")

df_pr_old.columns=["num", "datestr", "type", "header", "text", "url"]
df_pr_old = df_pr_old.drop(columns=['num'])
df_pr_old = df_pr_old.reset_index(drop=True)

# df_pr_old['date'] = [f"{i[0]}-{months[i[1]]}-{i[2]}" for i in df_new.datestr.str.split().tolist()]
df_pr_old['date'] = pd.to_datetime(df_pr_old['datestr'], format='%d.%m.%Y')
df_pr_old.head()

,datestr,type,header,text,url,date
0,29.01.2010,Пресс-релиз,О размещении средств федерального бюджета на б...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_154621...,2010-01-29
1,29.01.2010,Пресс-релиз,О проведении аукциона по предоставлению кредит...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_154621...,2010-01-29
2,29.01.2010,Пресс-релиз,О предварительном графике проведения аукционов...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_154618...,2010-01-29
3,29.01.2010,Пресс-релиз,Об условиях проведения Банком России 01.02.201...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_105415...,2010-01-29
4,29.01.2010,Пресс-релиз,Об условиях проведения Банком России 29.01.201...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_104722...,2010-01-29


## Объединяем часть 1 и 2

In [98]:
df = pd.concat([df_pr_old, df_new]).drop_duplicates().sort_values(by=['date', 'type']).reset_index(drop=True)
df = df[['date','type','header','text', 'url']]
df = df[(df.date.dt.date <= datetime.date(2022, 2, 1)) & (df.date.dt.date <= datetime.date(2022, 2, 1))]

In [88]:
l = [i.text for i in driver_base.find_elements(By.TAG_NAME, value='.referenceable')[1:] if i.text != "Архив выступлений"]
a = set(df[df.type == "Выступление"].header.tolist())

In [89]:
# df["rate"] = (df.type=="Пресс-релиз")&(df.header.str.find("ключев") > -1)
# df[(df.type=="Пресс-релиз")&(df.header.str.find("рефинансирования") > -1)]
df[(df.type=="Пресс-релиз")&(df.header.str.find("Банк России принял") > -1)]
df[(df.type=="Пресс-релиз")&(df.header.str.find("Банк России сохранил ключевую ставку") > -1)]
df[(df.header.str.find("Банк России сохранил ключевую ставку") > -1)]
df[df.date.dt.date == datetime.date(2015, 12, 11)]

,date,type,header,text,url
6306,2015-12-11,Выступление,Заявление Председателя Банка России Эльвиры На...,Сегодня Банк России принял решение сохранить к...,https://www.cbr.ru/press/event/?id=71
